In [57]:
import re
import pandas as pd
output_file_path = '/home/llama/Enrique/maderas/output.txt'
!python /home/llama/Enrique/maderas/yolov9/segment/predict.py \
--conf 0.2 --device 0 --max-det 1000 \
--weights /home/llama/Enrique/maderas/yolov9/runs/train-seg/gelan-c-seg3/weights/best.pt \
--source /home/llama/Enrique/maderas/yolov9/Wood-annotation-9/test/images/ > {output_file_path} 2>&1

In [58]:
# Read the output file
with open('output.txt', 'r') as file:
    output_text = file.read()

print(output_text) 

segment/predict: weights=['/home/llama/Enrique/maderas/yolov9/runs/train-seg/gelan-c-seg3/weights/best.pt'], source=/home/llama/Enrique/maderas/yolov9/Wood-annotation-9/test/images/, data=yolov9/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov9/runs/predict-seg, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1, retina_masks=False
YOLO 🚀 v0.1-104-g5b1ea9a Python-3.11.9 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24257MiB)

Fusing layers... 
gelan-c-seg summary: 414 layers, 27378319 parameters, 0 gradients, 144.3 GFLOPs
image 1/6 /home/llama/Enrique/maderas/yolov9/Wood-annotation-9/test/images/0030-1_jpg.rf.9d6568d8c612da31b2472577cea99103.jpg: 640x640 2 Anchura radio celulas 97s, 1 Depositos o gomas 

In [59]:
# Step 2: Read the output file and filter lines starting with "image"
with open(output_file_path, 'r') as file:
    lines = file.readlines()

image_lines = [line for line in lines if line.startswith("image")]

# Transform each line to keep only the image name and everything after "640x640"
transformed_lines = []
for line in image_lines:
    parts = line.split("640x640")
    if len(parts) > 1:
        image_name = parts[0].split('/')[-1].split(':')[0].strip()
        rest_of_line = parts[1].strip()
        transformed_line = f"{image_name}: {rest_of_line}"
        transformed_lines.append(transformed_line)

# Join the transformed lines back into a single string if needed for further processing
filtered_output_text = '\n'.join(transformed_lines)

filtered_output_path = '/home/llama/Enrique/maderas/filtered_output.txt'
with open(filtered_output_path, 'w') as file:
    file.write(filtered_output_text)

In [60]:
import json
import re

# Read the input file
file_path = 'filtered_output.txt'
with open(file_path, 'r') as file:
    input_data = file.readlines()

output = []

for line in input_data:
    # Step 1: Remove the Xms patterns (where X can be any number or a decimal number)
    cleaned_data = re.sub(r'\d+\.\d+ms', '', line)

    # Step 2: Split based on the rules provided
    file_name, attributes = cleaned_data.split(':')
    attributes_list = attributes.split(',')

    # Step 3: Process each attribute
    processed_attributes = []
    for attribute in attributes_list:
        attribute = attribute.strip()
        if attribute:
            if attribute.endswith('s'):
                processed_attributes.append(attribute[:-1])
            else:
                processed_attributes.append(attribute)

    # Create the final dictionary
    final_numbers = [attr.split()[-1] for attr in processed_attributes]
    final_numbers = [num if not num.isdigit() else str(int(num)) for num in final_numbers]
    final_numbers = ",".join(final_numbers)

    # Append to output list
    output.append({
        "name": file_name.strip(),
        "code": final_numbers
    })

# Save the result to a JSON file
output_file_path = 'output_processed.json'
with open(output_file_path, 'w') as f:
    json.dump(output, f, indent=4)

output, output_file_path




([{'name': '0030-1_jpg.rf.9d6568d8c612da31b2472577cea99103.jpg',
   'code': '97,58,78,80,81,83,V2,V3'},
  {'name': '0030-1_jpg.rf.ec7d31396dbb722c38bcdef111fad582.jpg',
   'code': '97,58,78,80,81,83,V2,V3'},
  {'name': '0030-6_jpg.rf.728e001481e01825292f263cef614194.jpg',
   'code': '97,58,80,81,82,83,V2,V3'},
  {'name': '0030-8_jpg.rf.327a00164ed7dce2d4998fe7964b6226.jpg',
   'code': '97,58,78,80,81,82,83,56,V2,V3'},
  {'name': '0294-12_jpg.rf.ee4a0d1c3a61b00b3be7dcef7f026a9a.jpg',
   'code': '96,97,89,3,56,V2'},
  {'name': '0550-4_png.rf.8a2c3c300657bdfe4f3458671f3b521c.jpg',
   'code': '99,6,8,3,56,V2'}],
 'output_processed.json')

In [65]:
import json
import pandas as pd

# Load JSON data
with open('output_processed.json', 'r') as f:
    json_data = json.load(f)

# Load Excel data
excel_file_path = '2024_05_13 Listado EUTR_codigos.xlsx'
df = pd.read_excel(excel_file_path)

# Prepare the data for comparison
codigos = df['Codigos '].astype(str).tolist()  # Note the space at the end of 'Codigos '
scientific_names = df['NOMBRE CIENTIFICO'].tolist()

# Function to get top 5 matches by number of matching elements
def get_top_matches_by_coincidence(input_code, codigos, scientific_names):
    input_set = set(input_code.split(','))
    match_counts = []
    
    for idx, code in enumerate(codigos):
        code_set = set(code.split(','))
        common_elements = input_set.intersection(code_set)
        match_counts.append((len(common_elements), scientific_names[idx], code))
    
    # Sort by number of matches in descending order and get top 5
    top_matches = sorted(match_counts, key=lambda x: x[0], reverse=True)[:5]
    return [(match[1], match[2], match[0]) for match in top_matches]

# Compare and predict top 5 candidates for each name in JSON
predictions = []
for item in json_data:
    name = item['name']
    code = item['code']
    top_matches = get_top_matches_by_coincidence(code, codigos, scientific_names)
    predictions.append({
        "name": name,
        "predictions": top_matches
    })

# Save the predictions to a new JSON file
output_predictions_path = 'coincidence_predictions.json'
with open(output_predictions_path, 'w') as f:
    json.dump(predictions, f, indent=4)

output_predictions_path

# Convert predictions to CSV format
csv_data = []
for pred in predictions:
    for match in pred['predictions']:
        csv_data.append([pred['name'], match[0], match[1], match[2]])

# Create a DataFrame and save to CSV
csv_df = pd.DataFrame(csv_data, columns=['Name', 'Predicted Scientific Name', 'Codigo', 'Match Count'])
csv_output_path = 'predictions.csv'
csv_df.to_csv(csv_output_path, index=False)

csv_output_path

'predictions.csv'